# Data Warehouse Medicare National QA - Member Enrollment Yearly

Performing QA on tables in dw_staging before moving them to data_warehouse schema

## Initialization

Just loading packages that will be used and initializing connection to GP DB.

In [1]:
import pandas as pd
import sys
import psycopg2
from tqdm.notebook import tqdm
sys.path.append('H:/uth_helpers')
from db_utils import get_dsn

In [14]:
connection = psycopg2.connect(get_dsn())
connection.autocommit = True

In [3]:
connection.close()

## Member Counts and Member Months

This table is an aggregate of the member_enrollment_monthly table where enrollment data is aggregated to a yearly level.

Because of this, we cannot compare the row count of the raw tables with this table since enrollment tables from raw data sources are usually not on a yearly level. Instead, we can try to compare the Member Months (MM) and other columns to see if the aggregrate logic implemented is working as intended

In [3]:
query = '''drop table if exists qa_reporting.dw_mcrn_mbr_enrl_yearly;
create table qa_reporting.dw_mcrn_mbr_enrl_yearly
(
    data_source text,
    calendar_year int,
    table_src text,
    dw_row_count int,
    dw_uth_mbr_id_count int,
    dw_src_mbr_id_count int,
    src_mbr_count int,
    mbr_count_diff int,
    mbr_count_percentage float,
    dw_mm int,
    src_mm int,
    mm_diff int,
    mm_diff_percentage float,
    date_generated date
);
'''

with connection.cursor() as cursor:
    cursor.execute(query)

In [4]:
with connection.cursor() as cursor:
      query = '''
insert into qa_reporting.dw_mcrn_mbr_enrl_yearly
(data_source, calendar_year, table_src, dw_row_count, dw_uth_mbr_id_count, dw_src_mbr_id_count, dw_mm,date_generated)
select data_source, 
        year, 
        table_id_src, 
        count(*),
        count(distinct uth_member_id),
        count(distinct member_id_src),
        sum(total_enrolled_months),
        now()::date
  from dw_staging.mcrn_member_enrollment_yearly
 group by 1,2,3;
      '''

      cursor.execute(query)


In [10]:
with connection.cursor() as cursor:
    
      query = '''
update qa_reporting.dw_mcrn_mbr_enrl_yearly a
set src_mbr_count = b.pat_count,
    mbr_count_diff = a.dw_src_mbr_id_count - b.pat_count,
    mbr_count_percentage = abs(a.dw_src_mbr_id_count - b.pat_count) / b.pat_count
from qa_reporting.medicare_national_counts b
where data_source = 'mcrn'
and a.calendar_year = b.year
and a.table_src = 'medicare_national.' || b.table_name
;
      '''

      cursor.execute(query)

      query = '''
with enrl_months as(
select year::int, bene_id, t.month_year_id as enrolled_month
            from medicare_national.mbsf_abcd_summary a
            cross join lateral (values (a.year || '01', a.mdcr_status_code_01), (a.year || '02', a.mdcr_status_code_02),
                              (a.year || '03', a.mdcr_status_code_03), (a.year || '04', a.mdcr_status_code_04), (a.year || '05', a.mdcr_status_code_05),
                              (a.year || '06', a.mdcr_status_code_06), (a.year || '07', a.mdcr_status_code_07), (a.year || '08', a.mdcr_status_code_08),
                              (a.year || '09', a.mdcr_status_code_09), (a.year || '10', a.mdcr_status_code_10), (a.year || '11', a.mdcr_status_code_11),
                              (a.year || '12', a.mdcr_status_code_12))
            t(month_year_id, enrollment_status)
            where t.enrollment_status in ('10','11','20','21','31')
),
enrl_my as (
    select 'medicare_national.mbsf_abcd_summary' table_name, year, count(enrolled_month) as mm
    from enrl_months
    group by 2
)
update qa_reporting.dw_mcrn_mbr_enrl_yearly a
set src_mm = b.mm,
    mm_diff = a.dw_mm - b.mm,
    mm_diff_percentage = abs(a.dw_mm - b.mm) / b.mm
from enrl_my b
where data_source = 'mcrn'
and a.calendar_year = b.year
and a.table_src = b.table_name
;
      '''

      cursor.execute(query)

In [11]:
df = pd.read_sql('''select * 
from qa_reporting.dw_mcrn_mbr_enrl_yearly 
order by mm_diff ;''', con=connection)
df

c:\Users\iperez9\.conda\envs\ip_env\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,data_source,calendar_year,table_src,dw_row_count,dw_uth_mbr_id_count,dw_src_mbr_id_count,src_mbr_count,mbr_count_diff,mbr_count_percentage,dw_mm,src_mm,mm_diff,mm_diff_percentage,date_generated
0,mcrn,2020,medicare_national.mbsf_abcd_summary,3363785,3363785,3363785,3382077,-18292,0.0,38566777,38770661,-203884,0.0,2023-05-30
1,mcrn,2018,medicare_national.mbsf_abcd_summary,3212639,3212639,3212639,3229338,-16699,0.0,36726840,36929927,-203087,0.0,2023-05-30
2,mcrn,2017,medicare_national.mbsf_abcd_summary,3116942,3116942,3116942,3133013,-16071,0.0,35627252,35827216,-199964,0.0,2023-05-30
3,mcrn,2016,medicare_national.mbsf_abcd_summary,3030708,3030708,3030708,3046466,-15758,0.0,34630907,34821230,-190323,0.0,2023-05-30
4,mcrn,2015,medicare_national.mbsf_abcd_summary,2947866,2947866,2947866,2963123,-15257,0.0,33629136,33818812,-189676,0.0,2023-05-30
5,mcrn,2019,medicare_national.mbsf_abcd_summary,3292969,3292969,3292969,3306928,-13959,0.0,37715342,37904124,-188782,0.0,2023-05-30
6,mcrn,2014,medicare_national.mbsf_abcd_summary,2861891,2861891,2861891,2877408,-15517,0.0,32640896,32819005,-178109,0.0,2023-05-30


Here we look at the overall difference of member enrollment and member count. We start of by just adding the counts for each of the source tables. However, due to how the yearly table is built, it may be better to look at member counts and member months as the data source overall.

In [12]:
df.groupby('calendar_year')[['mm_diff', 'mm_diff_percentage', 'mbr_count_diff', 'mbr_count_percentage']].sum()

,mm_diff,mm_diff_percentage,mbr_count_diff,mbr_count_percentage
calendar_year,,,,
2014,-178109,0.0,-15517,0.0
2015,-189676,0.0,-15257,0.0
2016,-190323,0.0,-15758,0.0
2017,-199964,0.0,-16071,0.0
2018,-203087,0.0,-16699,0.0
2019,-188782,0.0,-13959,0.0
2020,-203884,0.0,-18292,0.0


## Gender Counts

Just like with the monthly enrollment table, we need to check the counts and values of the other columns. Here we take a look at the gender_cd column

In [19]:
query = '''with mcrn_gen_cd as (
    select distinct year::int, bene_id, sex_ident_cd
    from medicare_national.mbsf_abcd_summary a
),
mcrn_gen as (
    select year, c.gender_cd, count(*) gender_count
    from mcrn_gen_cd m
    left join reference_tables.ref_gender c
    on c.data_source = 'mcr'
   and c.gender_cd_src = m.sex_ident_cd
    group by 1,2
), dw_gen as (
    select year, gender_cd, count(*) gender_count
    from dw_staging.mcrn_member_enrollment_yearly
    group by 1,2
)
select a.*, b.gender_count as src_gender_count, 
        a.gender_count - b.gender_count as gender_diff, 
        100. * abs(a.gender_count - b.gender_count) / b.gender_count as gender_diff_percent
from mcrn_gen b
full outer join dw_gen a
on a.year = b.year
and a.gender_cd = b.gender_cd
order by year, gender_cd;
'''

pd.read_sql(query,  con=connection)

,year,gender_cd,gender_count,src_gender_count,gender_diff,gender_diff_percent
0,2014,F,1566564,1574291,-7727,0.490824
1,2014,M,1295327,1303117,-7790,0.597797
2,2015,F,1613417,1621011,-7594,0.468473
3,2015,M,1334449,1342112,-7663,0.570966
4,2016,F,1658758,1666729,-7971,0.478242
5,2016,M,1371950,1379737,-7787,0.564383
6,2017,F,1706058,1714080,-8022,0.468006
7,2017,M,1410884,1418933,-8049,0.567257
8,2018,F,1760343,1768547,-8204,0.463884
9,2018,M,1452296,1460791,-8495,0.581534


## Plan Type

Check if plan types are properly mapped at a yearly level. If the plan type counts do not match between the member_enrollment_yearly table and the raw enrollment table, it most likely occurred when cleaning up the plan_type column. Ideally the count difference percent should be <= 1%.

In [50]:
query = '''with mcrn_enroll as (
    select year::int, bene_id, ent.plan_type, t.month_year_id
    from medicare_national.mbsf_abcd_summary a
    cross join lateral (values (year || '01', a.mdcr_entlmt_buyin_ind_01, a.mdcr_status_code_01), (year || '02', a.mdcr_entlmt_buyin_ind_02, a.mdcr_status_code_02),
                        (year || '03', a.mdcr_entlmt_buyin_ind_03, a.mdcr_status_code_03), (year || '04', a.mdcr_entlmt_buyin_ind_04, a.mdcr_status_code_04), (year || '05', a.mdcr_entlmt_buyin_ind_05, a.mdcr_status_code_05),
                        (year || '06', a.mdcr_entlmt_buyin_ind_06, a.mdcr_status_code_06), (year || '07', a.mdcr_entlmt_buyin_ind_07, a.mdcr_status_code_07), (year || '08', a.mdcr_entlmt_buyin_ind_08, a.mdcr_status_code_08),
                        (year || '09', a.mdcr_entlmt_buyin_ind_09, a.mdcr_status_code_09), (year || '10', a.mdcr_entlmt_buyin_ind_10, a.mdcr_status_code_10), (year || '11', a.mdcr_entlmt_buyin_ind_11, a.mdcr_status_code_11),
                        (year || '12', a.mdcr_entlmt_buyin_ind_12, a.mdcr_status_code_12))
    t(month_year_id, mcdcr_enrlmt, enrollment_status)
    join reference_tables.ref_medicare_entlmt_buyin ent 
    on ent.buyin_cd = t.mcdcr_enrlmt
    where t.enrollment_status in ('10','11','20','21','31')
),
mcrn_agg_enroll as (
    select year, bene_id, plan_type, max(month_year_id) my, count(*)
    from mcrn_enroll
    group by 1,2,3
),
mcrn_grp_enroll as (
    select *, row_number() over(partition by bene_id, year order by count desc, my desc) as my_grp
    from mcrn_agg_enroll
),
mcrn_plans as (
    select year, plan_type, count(*) plan_count
    from mcrn_grp_enroll 
    where my_grp = 1
    group by 1,2
),
dw_plans as (
    select year, plan_type, count(*) plan_count
    from dw_staging.mcrn_member_enrollment_yearly
    group by 1,2
)
select a.*, b.plan_count as src_plan_count, 
        a.plan_count - b.plan_count as plan_diff, 
        100. * abs(a.plan_count - b.plan_count) / b.plan_count as plan_diff_percent
from mcrn_plans b
join dw_plans a
on a.year = b.year
and a.plan_type = b.plan_type
order by year;
'''

plan_type_counts_df = pd.read_sql(query,  con=connection)
plan_type_counts_df

c:\Users\iperez9\.conda\envs\ip_env\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,year,plan_type,plan_count,src_plan_count,plan_diff,plan_diff_percent
0,2014,B,18026,18716,-690,3.686685
1,2014,A,245246,260337,-15091,5.796717
2,2014,AB,2598619,2598087,532,0.020477
3,2015,AB,2676022,2675377,645,0.024109
4,2015,B,18212,18880,-668,3.538136
5,2015,A,253632,268472,-14840,5.527578
6,2016,A,263894,278896,-15002,5.379066
7,2016,B,18143,18757,-614,3.273445
8,2016,AB,2748671,2748167,504,0.018339
9,2017,A,274594,290377,-15783,5.435348


In [51]:
plan_type_counts_df.groupby('year')['plan_diff_percent'].min(), plan_type_counts_df.groupby('year')['plan_diff_percent'].max()

(year
 2014    0.020477
 2015    0.024109
 2016    0.018339
 2017    0.030388
 2018    0.039668
 2019    0.043948
 2020    0.115824
 Name: plan_diff_percent, dtype: float64,
 year
 2014    5.796717
 2015    5.527578
 2016    5.379066
 2017    5.435348
 2018    5.509892
 2019    4.628536
 2020    4.630744
 Name: plan_diff_percent, dtype: float64)

In [52]:
plan_type_counts_df.sort_values('plan_diff_percent')

,year,plan_type,plan_count,src_plan_count,plan_diff,plan_diff_percent
8,2016,AB,2748671,2748167,504,0.018339
2,2014,AB,2598619,2598087,532,0.020477
3,2015,AB,2676022,2675377,645,0.024109
10,2017,AB,2824308,2823450,858,0.030388
12,2018,AB,2907794,2906641,1153,0.039668
17,2019,AB,2977558,2976250,1308,0.043948
19,2020,AB,3053692,3057233,-3541,0.115824
20,2020,B,18553,18922,-369,1.950111
15,2019,B,18986,19523,-537,2.750602
7,2016,B,18143,18757,-614,3.273445


In [54]:
plan_types_agg_df = plan_type_counts_df.groupby('year')['plan_count', 'src_plan_count', 'plan_diff'].sum()
plan_types_agg_df['percent_diff'] = 100. * plan_types_agg_df['plan_diff'] / plan_types_agg_df['src_plan_count'] 
plan_types_agg_df

C:\Users\iperez9\AppData\Local\Temp\3\ipykernel_11464\1871058508.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  plan_types_agg_df = plan_type_counts_df.groupby('year')['plan_count', 'src_plan_count', 'plan_diff'].sum()


,plan_count,src_plan_count,plan_diff,percent_diff
year,,,,
2014,2861891,2877140,-15249,-0.530005
2015,2947866,2962729,-14863,-0.501666
2016,3030708,3045820,-15112,-0.496155
2017,3116942,3132606,-15664,-0.500031
2018,3212639,3228969,-16330,-0.505734
2019,3292969,3306584,-13615,-0.411754
2020,3363785,3381851,-18066,-0.534204


## State

In [33]:
query = '''with mcrn_enroll as (
    select year::int, bene_id, case when state_cd is null then 'XX' else state_cd end as state
    from medicare_national.mbsf_abcd_summary m
    left outer join reference_tables.ref_medicare_state_codes e 
     on e.medicare_state_cd = m.state_code   
),
mcrn_states as (
    select year, state, count(*) state_count
    from mcrn_enroll m
    group by 1,2
), 
dw_plans as (
    select year, case when state is null then 'XX' else state end as state, count(*) state_count
    from dw_staging.mcrn_member_enrollment_yearly
    group by 1,2
)
select a.*, b.state_count as src_state_count, 
        a.state_count - b.state_count as plan_diff, 
        100. * abs(a.state_count - b.state_count) / b.state_count as plan_diff_percent
from mcrn_states b
full join dw_plans a
on a.year = b.year
and a.state = b.state
order by year;
'''

df = pd.read_sql(query,  con=connection)
df

c:\Users\iperez9\.conda\envs\ip_env\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,year,state,state_count,src_state_count,plan_diff,plan_diff_percent
0,2014,MS,28734,28831,-97,0.336443
1,2014,PA,132348,133036,-688,0.517153
2,2014,FL,204837,205702,-865,0.420511
3,2014,MT,10405,10441,-36,0.344795
4,2014,SD,8074,8100,-26,0.320988
...,...,...,...,...,...,...
373,2020,MA,73313,73589,-276,0.375056
374,2020,RI,12273,12308,-35,0.284368
375,2020,GA,95250,95523,-273,0.285795
376,2020,ND,7471,7488,-17,0.227030


We see that there is a large difference in state values assigned using DW logic compared to raw data. Ideally the difference should show that there are more values in the raw data compared to the values in DW due, especially for state values of 'XX'. When this happens, it means that the member now has a valid state value which can be used to group these member in future projects.

In [34]:
df.sort_values('plan_diff_percent', ascending=False)

,year,state,state_count,src_state_count,plan_diff,plan_diff_percent
25,2014,XX,22812,24248,-1436,5.922138
370,2020,TX,221814,230803,-8989,3.894663
99,2015,XX,23490,24205,-715,2.953935
146,2016,XX,24277,24916,-639,2.564617
277,2019,XX,26433,26915,-482,1.790823
...,...,...,...,...,...,...
349,2020,MS,32375,32438,-63,0.194217
366,2020,NC,108682,108889,-207,0.190102
340,2020,WV,23635,23678,-43,0.181603
363,2020,PR,39010,39078,-68,0.174011


Should get 50 states plus Puerto Rico (PR), Washington D.C. (DC), Virgin Islands (VI), and unknown (XX)

In [39]:
df['state'].unique()

array(['MS', 'PA', 'FL', 'MT', 'SD', 'IL', 'NV', 'MD', 'ND', 'HI', 'VA',
       'NE', 'DE', 'NC', 'GA', 'CO', 'MN', 'ME', 'SC', 'WA', 'LA', 'PR',
       'NH', 'DC', 'TX', 'XX', 'NY', 'MO', 'KY', 'TN', 'MI', 'AL', 'VT',
       'OH', 'AZ', 'NM', 'RI', 'CT', 'WY', 'IN', 'KS', 'MA', 'IA', 'WV',
       'CA', 'OR', 'UT', 'ID', 'AK', 'OK', 'WI', 'AR', 'VI', 'NJ'],
      dtype=object)

In [36]:
df['state'].unique().shape

(54,)

In [40]:
df[df['state'] == '']

,year,state,state_count,src_state_count,plan_diff,plan_diff_percent


In [41]:
df[df['state'].isna()]

,year,state,state_count,src_state_count,plan_diff,plan_diff_percent
